40 tipos de insectos

In [1]:
import math, re, os
import numpy as np
import tensorflow as tf
    
print("Tensorflow version " + tf.__version__)



2023-07-12 08:59:02.258004: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-12 08:59:02.259642: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-12 08:59:02.295935: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-12 08:59:02.296562: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-12 08:59:02.851159: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Tensorflow version 2.13.0


In [2]:
# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [3]:
species_folders = {
    'Dolycoris baccarum (Linnaeus)': 'carpetas/Dolycoris baccarum (Linnaeus)',
    'Lycorma delicatula (White)': 'carpetas/Lycorma delicatula (White)',
    'Eurydema dominulus (Scopoli)': 'carpetas/Eurydema dominulus (Scopoli)',
    'Pieris rapae (Linnaeus)':'carpetas/Pieris rapae (Linnaeus)',
    'Halyomorpha halys (Stål)':'carpetas/Halyomorpha halys (Stål)',
    'Spilosoma obliqua (Walker)':'carpetas/Spilosoma obliqua (Walker)',
    'Graphosoma rubrolineata (Westwood)':'carpetas/Graphosoma rubrolineata (Westwood)',
    'Luperomorpha suturalis (Chen)':'carpetas/Luperomorpha suturalis Chen',
    'Leptocorisa acuta (Thunberg)':'carpetas/Leptocorisa acuta (Thunberg)',
    'Sesamia inferens (Walker)':'carpetas/Sesamia inferens (Walker)',
    'Cicadella viridis (Linnaeus)':'carpetas/Cicadella viridis (Linnaeus)',
    'Callitettix versicolor (Fabricius)':'carpetas/Callitettix versicolor (Fabricius)',
    'Scotinophara lurida (Burmeister)':'carpetas/Scotinophara lurida (Burmeister)',
    'Cletus punctiger (Dallas)':'carpetas/Cletus punctiger (Dallas)',
    'Nezara viridula (Linnaeus)':'carpetas/Nezara viridula (Linnaeus)',
    'Dicladispa armigera (Olivier)':'carpetas/Dicladispa armigera (Olivier)',
    'Riptortus pedestris (Fabricius)':'carpetas/Riptortus pedestris (Fabricius)',
    'Maruca testulalis (Gryer)':'carpetas/Maruca testulalis Gryer',
    'Chauliops fallax (Scott)':'carpetas/Chauliops fallax Scott',
    'Chilo supperssalis (Walker)':'carpetas/Chilo supperssalis (Walker)',
    'Stollia ventralis (Westwood)':'carpetas/Stollia ventralis (Westwood)',
    'Nilaparvata lugens (Stål)':'carpetas/Nilaparvata lugens (Stål)',
    'Diostrombus politus (Uhler)':'carpetas/Diostrombus politus Uhler',
    'Phyllotreta striolata (Fabricius)':'carpetas/Phyllotreta striolata (Fabricius)',
    'Aulacophora indica (Gmelin)':'carpetas/Aulacophora indica (Gmelin)',
    'Laodelphax striatellus (Fallén)':'carpetas/Laodelphax striatellus (Fallén)',
    'Ceroplastes ceriferus (Anderson)':'carpetas/Ceroplastes ceriferus (Anderson)',
    'Corythucha marmorata(Uhler)':'carpetas/Corythucha marmorata(Uhler)',
    'Dryocosmus Kuriphilus (Yasumatsu)':'carpetas/Dryocosmus KuriphilusYasumatsu',
    'Porthesia taiwana (Shiraki)':'carpetas/Porthesia taiwana Shiraki',
    'Chromatomyia horticola(Goureau)':'carpetas/Chromatomyia horticola(Goureau)',
    'Iscadia inexacta (Walker, 1858)':'carpetas/Iscadia inexacta (Walker, 1858)',
    'Plutella xylostella (Linnaeus)':'carpetas/Plutella xylostella (Linnaeus)',
    'Empoasca flavescens (Fabricius)':'carpetas/Empoasca flavescens (Fabricius)',
    'Dolerus tritici (Chu)':'carpetas/Dolerus tritici Chu',
    'Spodoptera litura (Fabricius)':'carpetas/Spodoptera litura (Fabricius)',
    'Corythucha ciliata (Say)':'carpetas/Corythucha ciliata (Say)',
    'Bemisia tabaci (Gennadius)':'carpetas/Bemisia tabaci (Gennadius)',
    'Ceutorhynchus asper (Roelofs)':'carpetas/Ceutorhynchus asper Roelofs',
    'Strongyllodes variegatus (Fairmaire)':'carpetas/Strongyllodes variegatus (Fairmaire)',
}

In [4]:
IMAGE_SIZE = [512, 512]

AUTO = tf.data.experimental.AUTOTUNE

BATCH_SIZE = 32

CLASSES = species_folders.keys()

def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "id": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

In [7]:
def data_augment(image, label):
    # Thanks to the dataset.prefetch(AUTO)
    # statement in the next function (below), this happens essentially
    # for free on TPU. Data pipeline code is executed on the "CPU"
    # part of the TPU while the TPU itself is computing gradients.
    image = tf.image.random_flip_left_right(image)
    #image = tf.image.random_saturation(image, 0, 2)
    return image, label   

def get_training_dataset():
    dataset = load_dataset(species_folders.values(), labeled=True)
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_validation_dataset(ordered=False):
    dataset = load_dataset(species_folders.values(), labeled=True, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO)
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(species_folders.values(), labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec
    # files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

NUM_TRAINING_IMAGES = count_data_items(species_folders.values())
NUM_VALIDATION_IMAGES = count_data_items(species_folders.values())
NUM_TEST_IMAGES = count_data_items(species_folders.values())
print('Dataset: {} training images, {} validation images, {} unlabeled test images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))

AttributeError: 'NoneType' object has no attribute 'group'

In [5]:

dataset = []
image_paths = []
labels = []


In [6]:
for species_name, folder_path in species_folders.items():
    # Obtener las rutas de archivo de las imágenes en la carpeta
    species_images = glob.glob(folder_path + '/*.jpg')  # Ajusta la extensión según tus imágenes
    
    # Agregar las rutas de archivo y las etiquetas correspondientes a las listas
    image_paths.extend(species_images)
    labels.extend([species_name] * len(species_images))
    
    # Agregar las imágenes y las etiquetas a la lista de datasets
    dataset.extend(list(zip(species_images, [species_name] * len(species_images))))

NameError: name 'glob' is not defined

In [ ]:
for image_path, label in dataset:
    print(image_path, label)

carpetas/Dolycoris baccarum (Linnaeus)/003_4010.jpg Dolycoris baccarum (Linnaeus)
carpetas/Dolycoris baccarum (Linnaeus)/003_4013.jpg Dolycoris baccarum (Linnaeus)
carpetas/Dolycoris baccarum (Linnaeus)/DSC_2700.jpg Dolycoris baccarum (Linnaeus)
carpetas/Dolycoris baccarum (Linnaeus)/003_4018.jpg Dolycoris baccarum (Linnaeus)
carpetas/Dolycoris baccarum (Linnaeus)/DSC_2701.jpg Dolycoris baccarum (Linnaeus)
carpetas/Dolycoris baccarum (Linnaeus)/DSC_2697.jpg Dolycoris baccarum (Linnaeus)
carpetas/Dolycoris baccarum (Linnaeus)/AAA_1782_IJFR.jpg Dolycoris baccarum (Linnaeus)
carpetas/Dolycoris baccarum (Linnaeus)/DSC_7258_IJFR.jpg Dolycoris baccarum (Linnaeus)
carpetas/Dolycoris baccarum (Linnaeus)/003_4009.jpg Dolycoris baccarum (Linnaeus)
carpetas/Dolycoris baccarum (Linnaeus)/DSC_2690.jpg Dolycoris baccarum (Linnaeus)
carpetas/Dolycoris baccarum (Linnaeus)/AAA_1781_IJFR.jpg Dolycoris baccarum (Linnaeus)
carpetas/Dolycoris baccarum (Linnaeus)/DSC_2695.jpg Dolycoris baccarum (Linnaeus)
c

In [ ]:

dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

In [ ]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)


# Split the encoded labels into training and validation sets
train_labels = encoded_labels[:int(0.8 * len(encoded_labels))]
val_labels = encoded_labels[int(0.8 * len(encoded_labels)):]    

train_dataset = tf.data.Dataset.from_tensor_slices((image_paths[:int(0.8 * len(image_paths))], train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((image_paths[int(0.8 * len(image_paths)):], val_labels))

In [ ]:
def load_and_preprocess_image(image_path, label):
    # Leer la imagen desde el archivo
    image = tf.io.read_file(image_path)
    # Decodificar la imagen en un tensor
    image = tf.image.decode_jpeg(image, channels=3)
    # Redimensionar la imagen a un tamaño fijo
    image = tf.image.resize(image, (224, 224))
    # Normalizar los valores de píxeles en el rango [0, 1]
    image = image / 255.0
    
    return image, label

In [ ]:
dataset = dataset.map(load_and_preprocess_image)

In [ ]:
dataset = dataset.shuffle(len(image_paths))


train_dataset = train_dataset.map(load_and_preprocess_image)
train_dataset = train_dataset.shuffle(len(image_paths)).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.map(load_and_preprocess_image)
val_dataset = val_dataset.batch(32)

In [ ]:
from matplotlib import pyplot as plt

def batch_to_numpy_images_and_labels(data):
    images, labels = data
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()
    if numpy_labels.dtype == object: # binary string in this case,
                                     # these are image ID strings
        numpy_labels = [None for _ in enumerate(numpy_images)]
    # If no labels, only image IDs, return None for labels (this is
    # the case for test data)
    return numpy_images, numpy_labels

def title_from_label_and_target(label, correct_label):
    if correct_label is None:
        return CLASSES[label], True
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(CLASSES[label], 'OK' if correct else 'NO', u"\u2192" if not correct else '',
                                CLASSES[correct_label] if not correct else ''), correct

def display_one_insect(image, title, subplot, red=False, titlesize=16):
    plt.subplot(*subplot)
    plt.axis('off')
    plt.imshow(image)
    if len(title) > 0:
        plt.title(title, fontsize=int(titlesize) if not red else int(titlesize/1.2), color='red' if red else 'black', fontdict={'verticalalignment':'top'})
    return (subplot[0], subplot[1], subplot[2]+1)
    
def display_batch_of_images(databatch, predictions=None):
    """This will work with:
    display_batch_of_images(images)
    display_batch_of_images(images, predictions)
    display_batch_of_images((images, labels))
    display_batch_of_images((images, labels), predictions)
    """
    # data
    images, labels = batch_to_numpy_images_and_labels(databatch)
    if labels is None:
        labels = [None for _ in enumerate(images)]
        
    # auto-squaring: this will drop data that does not fit into square
    # or square-ish rectangle
    rows = int(math.sqrt(len(images)))
    cols = len(images)//rows
        
    # size and spacing
    FIGSIZE = 13.0
    SPACING = 0.1
    subplot=(rows,cols,1)
    if rows < cols:
        plt.figure(figsize=(FIGSIZE,FIGSIZE/cols*rows))
    else:
        plt.figure(figsize=(FIGSIZE/rows*cols,FIGSIZE))
    
    # display
    for i, (image, label) in enumerate(zip(images[:rows*cols], labels[:rows*cols])):
        title = '' if label is None else CLASSES[label]
        correct = True
        if predictions is not None:
            title, correct = title_from_label_and_target(predictions[i], label)
        dynamic_titlesize = FIGSIZE*SPACING/max(rows,cols)*40+3 # magic formula tested to work from 1x1 to 10x10 images
        subplot = display_one_insect(image, title, subplot, not correct, titlesize=dynamic_titlesize)
    
    #layout
    plt.tight_layout()
    if label is None and predictions is None:
        plt.subplots_adjust(wspace=0, hspace=0)
    else:
        plt.subplots_adjust(wspace=SPACING, hspace=SPACING)
    plt.show()


def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])  

In [ ]:
ds_iter = iter(train_dataset.unbatch().batch(20))

In [ ]:
#one_batch = next(ds_iter)
#display_batch_of_images(one_batch)

In [ ]:
pretrained_base = keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

pretrained_base.trainable = False

In [ ]:
model = keras.Sequential([
    pretrained_base,
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(species_folders), activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(train_dataset, validation_data=val_dataset, epochs=10)

Epoch 1/10


100/100 [==============================] - 98s 968ms/step - loss: 2.1567 - accuracy: 0.4453 - val_loss: 10.6315 - val_accuracy: 0.0125
Epoch 2/10
100/100 [==============================] - 97s 965ms/step - loss: 0.6217 - accuracy: 0.8743 - val_loss: 13.9333 - val_accuracy: 0.0150
Epoch 3/10
100/100 [==============================] - 96s 963ms/step - loss: 0.2825 - accuracy: 0.9514 - val_loss: 15.9074 - val_accuracy: 0.0163
Epoch 4/10
100/100 [==============================] - 96s 961ms/step - loss: 0.1530 - accuracy: 0.9812 - val_loss: 17.2335 - val_accuracy: 0.0163
Epoch 5/10
100/100 [==============================] - 96s 959ms/step - loss: 0.0918 - accuracy: 0.9928 - val_loss: 18.3070 - val_accuracy: 0.0163
Epoch 6/10
100/100 [==============================] - 97s 965ms/step - loss: 0.0606 - accuracy: 0.9966 - val_loss: 18.9557 - val_accuracy: 0.0163
Epoch 7/10
100/100 [==============================] - 96s 963ms/step - loss: 0.0405 - accuracy: 0.9978 - val_loss: 19.7805 - val_accura

In [ ]:
test_loss, test_acc = model.evaluate(val_dataset)
print('Precisión en el conjunto de prueba:', test_acc)

25/25 [==============================] - 20s 781ms/step - loss: 21.3836 - accuracy: 0.0163
Precisión en el conjunto de prueba: 0.016290726140141487


In [ ]:
model.save('species_recognition_model.h5')